In [1]:
from langchain_community.graphs import Neo4jGraph
import os
from dotenv import load_dotenv
from typing import List
from langchain_groq import ChatGroq
import os
load_dotenv()


True

In [2]:
HF_TOKEN=os.getenv("HF_TOKEN")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [3]:
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

In [4]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

C:\Users\Welcome\AppData\Local\Temp\ipykernel_1276\2244984801.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)


In [5]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')



In [6]:

def embed_text(text:str)->List:
    embeddings = embed_model.encode(text)
    return embeddings.tolist()

In [47]:
query="Suggest some movies on patriotism"

In [48]:
query_embed=embed_text(query)

In [49]:
query_embed

[-0.023437269032001495,
 0.06710079312324524,
 0.013912674970924854,
 -0.015838762745261192,
 0.015769435092806816,
 0.0030142366886138916,
 -0.006137921940535307,
 -0.014090355485677719,
 -0.011048924177885056,
 0.008511359803378582,
 -0.0021139979362487793,
 0.012394534423947334,
 0.01777706667780876,
 -0.043942373245954514,
 -0.02659684605896473,
 -0.02748909592628479,
 0.04303605481982231,
 0.021015189588069916,
 0.037352923303842545,
 -0.0404166653752327,
 -0.035106394439935684,
 -0.00695391371846199,
 0.0011015386553481221,
 -0.038910556584596634,
 -0.01626121997833252,
 0.0019849345553666353,
 0.04783707484602928,
 -0.026999235153198242,
 0.0395561084151268,
 -0.05471205711364746,
 0.038090210407972336,
 -0.06380251049995422,
 0.015391632914543152,
 -0.010495328344404697,
 1.0986747156493948e-06,
 -0.005394411738961935,
 0.03963695839047432,
 -0.06713593751192093,
 0.045445725321769714,
 0.027145251631736755,
 0.00258436007425189,
 0.06478506326675415,
 -0.0006767633021809161,
 

## Performing Similarity Search

In [50]:
result = graph.query("""
    with $query_embed as question_embedding      // Use the provided question embedding as 'question_embedding'
    CALL db.index.vector.queryNodes(                    // Call the vector index query function
        'movie_tagline_embeddings',                     // Name of the vector index to query against
        $top_k,                                         // Number of top results to retrieve
        question_embedding                              // The question embedding to compare against
        ) YIELD node AS movie, score                    // Yield each matched node and its similarity score
    RETURN movie.title,movie.genre,movie.imdbRating,movie.tagline, score            // Return the title, tagline, and similarity score of each movie
    """,
    params={
        "query_embed": query_embed,       # Pass the question embedding as a parameter
        "top_k": 3                                     # Specify the number of top results to retrieve
    })
result

[#D1D4]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.124.169.171', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 0.9972958481121731s (Unable to retrieve routing information)
[#D079]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-effe28b5-9797.production-orch-0703.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687))): OSError('No data')
Transaction failed and will be retried in 1.9577589208284776s (Failed to read from defunct connection IPv4Address(('si-effe28b5-9797.production-orch-0703.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687))))
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property

[{'movie.title': 'Kesari',
  'movie.genre': None,
  'movie.imdbRating': 7.4,
  'movie.tagline': 'Bravery beyond borders',
  'score': 0.7190485000610352},
 {'movie.title': 'Bajirao Mastani',
  'movie.genre': None,
  'movie.imdbRating': 7.5,
  'movie.tagline': 'A Romance of a Warrior.',
  'score': 0.6888480186462402},
 {'movie.title': 'Veer-Zaara',
  'movie.genre': None,
  'movie.imdbRating': 7.9,
  'movie.tagline': 'Crossing borders, winning hearts',
  'score': 0.6709871292114258}]

## Adding LLM for final answer processing

In [51]:
prompt = f"# Question:\n{query}\n\n#  Graph DB Search Results:\n{result}"

In [52]:
messages = [
    {"role": "system", "content": str(
        "You will be given the user question along with the search result of that question over a Neo4j graph database. Give the user the proper answer in a helpful tone."
    )},
    {"role": "user", "content": prompt}
]

In [56]:
llm=ChatGroq(model="llama-3.3-70b-versatile")

In [57]:
print(llm.invoke(messages).content)

It seems like you're in the mood for some patriotic movies. I've got some great suggestions for you based on our search results. Here are a few movies that might spark your sense of national pride:

1. **Kesari** - This movie has a great tagline, "Bravery beyond borders," and an impressive IMDB rating of 7.4. It's definitely worth checking out if you're looking for a story that showcases courage and patriotism.
2. **Bajirao Mastani** - With an IMDB rating of 7.5, this movie seems like a great choice for anyone who loves historical dramas with a touch of romance. The tagline, "A Romance of a Warrior," suggests that it's a powerful and emotional story.
3. **Veer-Zaara** - This movie has an impressive IMDB rating of 7.9 and a beautiful tagline, "Crossing borders, winning hearts." It's a classic tale of love and patriotism that's sure to leave you feeling inspired.

All of these movies seem like great options if you're looking for something that will make you feel proud and connected to yo